# Binary Golf Course Classifier

## Architecture Overview

This notebook implements a **binary classifier** using **MobileNetV2 transfer learning** to distinguish golf course imagery from non-golf imagery.

### Why MobileNetV2?
- **Lightweight**: Depthwise separable convolutions reduce parameters by ~9x vs standard convs
- **Fast inference**: Designed for mobile/edge deployment
- **Strong features**: ImageNet pretrained weights generalize well to aerial imagery
- **Inverted residuals**: Linear bottlenecks preserve information flow

### Transfer Learning Strategy
1. **Freeze base model**: Use pretrained MobileNetV2 as fixed feature extractor
2. **Custom head**: Add GlobalAveragePooling + Dense layers for binary classification
3. **Fine-tune**: Train only the classification head initially

### Dataset Strategy
Combines two datasets for robust training:
- **Danish Golf Courses**: High-resolution orthophotos (positive class)
- **UC Merced Land Use**: 21 land use classes including golf (mixed positive/negative)

In [ ]:
# Environment detection for Colab/local compatibility
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab")
    !pip install -q kagglehub datasets
else:
    print("Running locally")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from datasets import load_dataset
import kagglehub

## GPU Configuration

Mixed precision (float16) accelerates training on modern GPUs while maintaining accuracy.

In [ ]:
print(f"TensorFlow version: {tf.__version__}")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    keras.mixed_precision.set_global_policy('mixed_float16')
    print(f"GPU configured: {len(gpus)} device(s)")
else:
    print("No GPU detected, using CPU")

## Hyperparameters

| Parameter | Value | Rationale |
|-----------|-------|----------|
| Image Size | 224×224 | MobileNetV2 default input size |
| Batch Size | 32 | Good balance of speed and gradient stability |
| Learning Rate | 1e-4 | Conservative for transfer learning |

In [ ]:
IMAGE_SIZE = (224, 224)  # MobileNetV2 default input
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
MAX_EPOCHS = 15

OUTPUT_DIR = '/content/output' if IN_COLAB else './output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Dataset Loading

### Data Sources
1. **UC Merced Land Use Dataset** (HuggingFace)
   - 2100 images across 21 land use classes
   - 100 images per class, 256×256 pixels
   - Class 9 = Golf courses

2. **Danish Golf Courses** (Kaggle)
   - High-resolution orthophotos
   - All positive (golf course) samples

In [ ]:
print("Loading UC Merced dataset...")
ucmerced = load_dataset("blanchon/UC_Merced", split="train")
print(f"UC Merced: {len(ucmerced)} images, 21 classes")

print("Loading Danish Golf Course dataset...")
golf_dataset_path = kagglehub.dataset_download('jacotaco/danish-golf-courses-orthophotos')
IMAGES_DIR = os.path.join(golf_dataset_path, '1. orthophotos')
danish_golf_files = [os.path.join(IMAGES_DIR, f) for f in os.listdir(IMAGES_DIR)]
print(f"Danish golf: {len(danish_golf_files)} images")

## Negative Class Strategy

Not all non-golf classes are equally useful for training. We split negatives into:

### Challenging Negatives (Visually Similar to Golf)
These classes share visual features with golf courses:
- **Baseball diamond**: Green grass, maintained playing surface
- **Tennis court**: Rectangular playing area
- **Agricultural**: Large green areas from above
- **Residential**: Mix of grass and structures
- **River**: Water features (similar to water hazards)
- **Chaparral**: Natural green vegetation

### Easy Negatives (Clearly Different)
Obvious non-golf imagery:
- Airplanes, buildings, freeways, harbors, parking lots, etc.

### Why This Matters
- Training on only easy negatives leads to poor generalization
- Challenging negatives force the model to learn discriminative features
- Stratified splits ensure both types appear in train/val/test

In [ ]:
# UC Merced class definitions
# Challenging: visually similar to golf courses
CHALLENGING_CLASSES = {
    2: 'baseballdiamond',   # Green grass, maintained surface
    20: 'tenniscourt',      # Rectangular playing area
    0: 'agricultural',      # Large green areas
    12: 'mediumresidential', # Mix of grass and structures
    18: 'sparseresidential', # Similar to above
    6: 'denseresidential',   # Urban with some green
    16: 'river',            # Water features
    5: 'chaparral',         # Natural vegetation
}

# Easy: clearly different from golf courses
EASY_CLASSES = {
    1: 'airplane', 3: 'beach', 4: 'buildings', 7: 'forest',
    8: 'freeway', 10: 'harbor', 11: 'intersection', 13: 'mobilehomepark',
    14: 'overpass', 15: 'parkinglot', 17: 'runway', 19: 'storagetanks',
}

# Golf course class (UC Merced label 9)
ucmerced_golf_samples = [item for item in ucmerced if item['label'] == 9]
print(f"UC Merced golf: {len(ucmerced_golf_samples)} images")

## Data Augmentation

Augmentation prevents overfitting and improves generalization:
- **Horizontal flip**: Golf courses look similar flipped
- **Rotation**: Small rotations (±5°) for viewpoint invariance
- **Zoom**: Simulates different altitudes
- **Brightness/Contrast**: Handles varying lighting conditions

In [ ]:
def get_augmentation_layer():
    """Keras Sequential augmentation pipeline."""
    return keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.05),      # ±5° rotation
        layers.RandomZoom(0.1),           # ±10% zoom
        layers.RandomBrightness(0.1),     # ±10% brightness
        layers.RandomContrast(0.1),       # ±10% contrast
    ], name='augmentation')

## Image Loading

All images normalized to [0, 1] range and resized to 224×224.

In [ ]:
def prepare_danish_golf_images(image_paths, target_size=(224, 224)):
    """Load Danish golf orthophotos."""
    images = []
    for img_path in image_paths:
        img = Image.open(img_path).convert('RGB')
        img = img.resize(target_size)
        img_array = np.array(img) / 255.0  # Normalize to [0, 1]
        images.append(img_array)
    return np.array(images, dtype=np.float32)


def prepare_ucmerced_golf_images(golf_samples, target_size=(224, 224)):
    """Load UC Merced golf class images."""
    images = []
    for sample in golf_samples:
        img = sample['image'].resize(target_size)
        img_array = np.array(img) / 255.0
        images.append(img_array)
    return np.array(images, dtype=np.float32)


def prepare_ucmerced_negatives_by_difficulty(dataset, target_size=(224, 224)):
    """Split UC Merced negatives into challenging and easy."""
    challenging_images = []
    easy_images = []

    for class_id in CHALLENGING_CLASSES.keys():
        class_samples = [item for item in dataset if item['label'] == class_id]
        for sample in class_samples:
            img = sample['image'].resize(target_size)
            img_array = np.array(img) / 255.0
            challenging_images.append(img_array)

    for class_id in EASY_CLASSES.keys():
        class_samples = [item for item in dataset if item['label'] == class_id]
        for sample in class_samples:
            img = sample['image'].resize(target_size)
            img_array = np.array(img) / 255.0
            easy_images.append(img_array)

    return np.array(challenging_images, dtype=np.float32), np.array(easy_images, dtype=np.float32)

In [ ]:
# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Loading images...")
danish_golf_images = prepare_danish_golf_images(danish_golf_files, IMAGE_SIZE)
ucmerced_golf_images = prepare_ucmerced_golf_images(ucmerced_golf_samples, IMAGE_SIZE)
challenging_negatives, easy_negatives = prepare_ucmerced_negatives_by_difficulty(ucmerced, IMAGE_SIZE)

print(f"Positives: {len(danish_golf_images) + len(ucmerced_golf_images)} (Danish: {len(danish_golf_images)}, UC Merced: {len(ucmerced_golf_images)})")
print(f"Negatives: {len(challenging_negatives) + len(easy_negatives)} (Challenging: {len(challenging_negatives)}, Easy: {len(easy_negatives)})")

## Stratified Data Splitting

Different split ratios per data source ensure balanced representation:

| Source | Train | Val | Test | Rationale |
|--------|-------|-----|------|-----------|
| Danish Golf | 80% | 10% | 10% | Primary positive source |
| UC Merced Golf | 60% | 20% | 20% | Supplement positives |
| Challenging Negatives | 50% | 30% | 20% | Heavy validation focus |
| Easy Negatives | 70% | 20% | 10% | Less emphasis needed |

This ensures:
- Validation set is harder (more challenging negatives)
- Test set reflects real-world distribution
- Training sees all difficulty levels

In [ ]:
# Stratified splits for each data source
# Danish golf: 80/10/10
n_danish = len(danish_golf_images)
danish_indices = np.random.permutation(n_danish)
danish_train_idx = danish_indices[:int(0.80 * n_danish)]
danish_val_idx = danish_indices[int(0.80 * n_danish):int(0.90 * n_danish)]
danish_test_idx = danish_indices[int(0.90 * n_danish):]

# UC Merced golf: 60/20/20
n_ucm_golf = len(ucmerced_golf_images)
ucm_golf_indices = np.random.permutation(n_ucm_golf)
ucm_golf_train_idx = ucm_golf_indices[:int(0.60 * n_ucm_golf)]
ucm_golf_val_idx = ucm_golf_indices[int(0.60 * n_ucm_golf):int(0.80 * n_ucm_golf)]
ucm_golf_test_idx = ucm_golf_indices[int(0.80 * n_ucm_golf):]

# Challenging negatives: 50/30/20 (more in val to test generalization)
n_challenging = len(challenging_negatives)
challenging_indices = np.random.permutation(n_challenging)
challenging_train_idx = challenging_indices[:int(0.50 * n_challenging)]
challenging_val_idx = challenging_indices[int(0.50 * n_challenging):int(0.80 * n_challenging)]
challenging_test_idx = challenging_indices[int(0.80 * n_challenging):]

# Easy negatives: 70/20/10
n_easy = len(easy_negatives)
easy_indices = np.random.permutation(n_easy)
easy_train_idx = easy_indices[:int(0.70 * n_easy)]
easy_val_idx = easy_indices[int(0.70 * n_easy):int(0.90 * n_easy)]
easy_test_idx = easy_indices[int(0.90 * n_easy):]

In [ ]:
# Combine all splits
# Label: 1 = Golf, 0 = Not Golf
train_images = np.concatenate([
    danish_golf_images[danish_train_idx],
    ucmerced_golf_images[ucm_golf_train_idx],
    challenging_negatives[challenging_train_idx],
    easy_negatives[easy_train_idx]
])
train_labels = np.concatenate([
    np.ones(len(danish_train_idx)),    # Golf = 1
    np.ones(len(ucm_golf_train_idx)),
    np.zeros(len(challenging_train_idx)),  # Not Golf = 0
    np.zeros(len(easy_train_idx))
])

val_images = np.concatenate([
    danish_golf_images[danish_val_idx],
    ucmerced_golf_images[ucm_golf_val_idx],
    challenging_negatives[challenging_val_idx],
    easy_negatives[easy_val_idx]
])
val_labels = np.concatenate([
    np.ones(len(danish_val_idx)),
    np.ones(len(ucm_golf_val_idx)),
    np.zeros(len(challenging_val_idx)),
    np.zeros(len(easy_val_idx))
])

test_images = np.concatenate([
    danish_golf_images[danish_test_idx],
    ucmerced_golf_images[ucm_golf_test_idx],
    challenging_negatives[challenging_test_idx],
    easy_negatives[easy_test_idx]
])
test_labels = np.concatenate([
    np.ones(len(danish_test_idx)),
    np.ones(len(ucm_golf_test_idx)),
    np.zeros(len(challenging_test_idx)),
    np.zeros(len(easy_test_idx))
])

# Shuffle each split
train_idx = np.random.permutation(len(train_images))
train_images, train_labels = train_images[train_idx], train_labels[train_idx]

val_idx = np.random.permutation(len(val_images))
val_images, val_labels = val_images[val_idx], val_labels[val_idx]

test_idx = np.random.permutation(len(test_images))
test_images, test_labels = test_images[test_idx], test_labels[test_idx]

print(f"Train: {len(train_images)} (Golf: {int(train_labels.sum())}, Not Golf: {int(len(train_labels) - train_labels.sum())})")
print(f"Val: {len(val_images)} (Golf: {int(val_labels.sum())}, Not Golf: {int(len(val_labels) - val_labels.sum())})")
print(f"Test: {len(test_images)} (Golf: {int(test_labels.sum())}, Not Golf: {int(len(test_labels) - test_labels.sum())})")

## TensorFlow Dataset Pipeline

Efficient data loading with:
- **Shuffling**: Randomize batch composition each epoch
- **Batching**: Group samples for parallel processing
- **Prefetching**: Load next batch while GPU processes current
- **Stochastic augmentation**: 50% chance to augment each batch

In [ ]:
augmentation_layer = get_augmentation_layer()

def augment_with_passthrough(images, labels):
    """50% chance to apply augmentation to batch."""
    def apply_aug():
        return tf.cast(augmentation_layer(images, training=True), tf.float32)
    def keep_orig():
        return tf.cast(images, tf.float32)
    should_aug = tf.random.uniform([]) >= 0.5
    return tf.cond(should_aug, apply_aug, keep_orig), labels

# Training: shuffle + augment + batch + prefetch
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.shuffle(1000, reshuffle_each_iteration=True)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.map(augment_with_passthrough, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

# Validation/Test: no augmentation
val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

## Model Architecture

```
Input (224×224×3)
    │
    ▼
┌─────────────────────────────────────┐
│      MobileNetV2 (frozen)           │
│  - 53 layers, 3.4M parameters       │
│  - Depthwise separable convolutions │
│  - Inverted residual blocks         │
│  - Output: 7×7×1280 feature map     │
└─────────────────────────────────────┘
    │
    ▼
┌─────────────────────────────────────┐
│  GlobalAveragePooling2D             │
│  - Reduces 7×7×1280 → 1280          │
│  - Spatial invariance               │
└─────────────────────────────────────┘
    │
    ▼
┌─────────────────────────────────────┐
│  Classification Head                │
│  - Dropout(0.3)                     │
│  - Dense(128, relu)                 │
│  - Dropout(0.2)                     │
│  - Dense(1, sigmoid)                │
└─────────────────────────────────────┘
    │
    ▼
Output: P(golf) ∈ [0, 1]
```

### Key Design Choices
- **Frozen base**: Pretrained features are already excellent for aerial imagery
- **GlobalAveragePooling**: Better generalization than Flatten (fewer parameters)
- **Dropout**: Regularization to prevent overfitting on small dataset
- **Sigmoid output**: Binary probability, threshold at 0.5

In [ ]:
def build_golf_classifier(input_shape=(224, 224, 3)):
    """MobileNetV2-based binary classifier."""
    # Load pretrained MobileNetV2 (frozen)
    base_model = keras.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,  # Remove classification head
        weights='imagenet'
    )
    base_model.trainable = False  # Freeze all layers

    inputs = keras.Input(shape=input_shape)
    
    # MobileNetV2 expects input in [-1, 1] range
    x = keras.applications.mobilenet_v2.preprocess_input(inputs * 255.0)
    
    # Feature extraction
    x = base_model(x, training=False)
    
    # Classification head
    x = layers.GlobalAveragePooling2D()(x)  # 7×7×1280 → 1280
    x = layers.Dropout(0.3)(x)              # Regularization
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    # Binary output with float32 for numerical stability
    outputs = layers.Dense(1, activation='sigmoid', dtype='float32')(x)

    return keras.Model(inputs=inputs, outputs=outputs, name='GolfClassifier')


model = build_golf_classifier(input_shape=(*IMAGE_SIZE, 3))

# Binary crossentropy loss for binary classification
# Track precision and recall in addition to accuracy
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

model.summary()

## Training Callbacks

- **ModelCheckpoint**: Save best model by validation accuracy
- **EarlyStopping**: Stop if no improvement for 7 epochs
- **ReduceLROnPlateau**: Halve LR if plateau for 4 epochs

In [ ]:
callback_list = [
    callbacks.ModelCheckpoint(
        filepath=os.path.join(OUTPUT_DIR, 'best_golf_classifier.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=7,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=4,
        verbose=1,
        min_lr=1e-7
    )
]

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=MAX_EPOCHS,
    callbacks=callback_list,
    verbose=1
)

## Evaluation Metrics

For binary classification, accuracy alone can be misleading. We also track:

- **Precision**: Of predicted golf, what % is actually golf? (avoid false positives)
- **Recall**: Of actual golf, what % did we find? (avoid false negatives)
- **F1 Score**: Harmonic mean of precision and recall (balanced metric)

In [ ]:
print("Validation:")
val_loss, val_acc, val_prec, val_rec = model.evaluate(val_ds, verbose=0)
val_f1 = 2 * (val_prec * val_rec) / (val_prec + val_rec) if (val_prec + val_rec) > 0 else 0
print(f"  Accuracy: {val_acc:.4f}, Precision: {val_prec:.4f}, Recall: {val_rec:.4f}, F1: {val_f1:.4f}")

print("Test:")
test_loss, test_acc, test_prec, test_rec = model.evaluate(test_ds, verbose=0)
test_f1 = 2 * (test_prec * test_rec) / (test_prec + test_rec) if (test_prec + test_rec) > 0 else 0
print(f"  Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}, Recall: {test_rec:.4f}, F1: {test_f1:.4f}")

In [ ]:
model.save(os.path.join(OUTPUT_DIR, 'final_golf_classifier.keras'))
print(f"Model saved to {OUTPUT_DIR}")

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Val')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'training_history.png'), dpi=150)
plt.show()

## Sample Predictions

Visualize model predictions on validation samples.

In [ ]:
sample_indices = np.random.choice(len(val_images), size=6, replace=False)

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

for i, idx in enumerate(sample_indices):
    img = val_images[idx]
    true_label = val_labels[idx]
    pred_prob = model.predict(np.expand_dims(img, axis=0), verbose=0)[0][0]
    pred_label = 1 if pred_prob > 0.5 else 0

    axes[i].imshow(img)
    axes[i].set_title(
        f"True: {'Golf' if true_label == 1 else 'Not Golf'}\n"
        f"Pred: {'Golf' if pred_label == 1 else 'Not Golf'} ({pred_prob:.1%})",
        color='green' if pred_label == true_label else 'red'
    )
    axes[i].axis('off')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'sample_predictions.png'), dpi=150)
plt.show()

In [ ]:
# Download trained model (Colab only)
if IN_COLAB:
    from google.colab import files
    files.download(os.path.join(OUTPUT_DIR, 'final_golf_classifier.keras'))